# Einfache Quantilplots

## Hilfsfunktionen

In [ ]:
library(ggplot2)
library(patchwork)

In [ ]:
single_plot <- function(data, color, title, aspect_ratio=1) {
  gg <- ggplot(data, aes(x = x, y = y)) +
      geom_line(color = color, linewidth=1) +
      labs(x = NULL, y = NULL) +   
      ggtitle(title) +
      theme_bw() +    
      theme(plot.title = element_text(size=20, color = "black", hjust = 0.5), 
            axis.text = element_text(size = 12, color = "black"),
            aspect.ratio=aspect_ratio)
}

In [ ]:
combined_plot <- function(x, dfunc, pfunc, qfunc, distribution_name) {
  pdf_data  <- data.frame(x = x, y = dfunc(x))
  pdf_graph <- single_plot(pdf_data, "blue", "Dichte")

  cdf_data  <- data.frame(x = x, y = pfunc(x))
  cdf_graph <- single_plot(cdf_data, "green", "Verteilung")

  p <- seq(0, 1, length=100)
  quantiles  <- data.frame(x = p, y = qfunc(p))
  quantile_graph <- single_plot(quantiles, "red", "Quantilplot")

  (pdf_graph | cdf_graph | quantile_graph) +
    plot_annotation( title = distribution_name, theme = theme(plot.title = element_text(size = 30, hjust = 0.5) ) )
}

## Beispiel 1 : Standardnormalverteilung

In [ ]:
x <- seq(-3, 3, length=100)
combined_plot(x, dnorm, pnorm, qnorm, "Normalverteilung")

## Beispiel 2 : Chi-Quadrat-Verteilung - Rechtsschief

In [ ]:
df <- 5

pdf_func <- function(x) {
  dchisq(x = x, df=df)
}
cdf_func <- function(x) {
  pchisq(q = x, df=df)
}
quantil_func <- function(x) {
  qchisq(p = x, df=df)
}

x <- seq(0, 20, length=100)
combined_plot(x, pdf_func, cdf_func, quantil_func, "Chi-Quadrat-Verteilung")

## Beispiel 3 : Bimodale Mischverteilung 

s.a. Oldford - Figure 3 (iv)
$$
Y \sim \frac{1}{2} N(0,1) + \frac{1}{2} N(6,1)
$$

Simulation Mischverteilung

In [ ]:
mu1 <- 0
sigma1 <- 1
mu2 <- 6
sigma2 <- 1

a <- 0.5
b <- 0.5

n <- 100000
set.seed(123)
component1 <- rnorm(n, mean = mu1, sd = sigma1)
component2 <- rnorm(n, mean = mu2, sd = sigma2)
mixture_data <- ifelse(runif(n) < 0.5, component1, component2) # generiere Werte aus [0,1] der Gleichverteilung

Empirische Dichtefunktion

In [ ]:
density_estimate <- density(mixture_data)
empirical_pdf <- approxfun(density_estimate$x, density_estimate$y)

Empirische Verteilungsfunktion

In [ ]:
empirical_cdf = ecdf(mixture_data)

Auswertung PDF und CDF

In [ ]:
x <- seq(-3, 10, length=100)
pdf <- empirical_pdf(x)
# alternativ : pdf <- a * dnorm(x, mean = mu1, sd = sigma1) + b * dnorm(x, mean = mu2, sd = sigma2)
cdf <- empirical_cdf(x)
# alternativ : cdf <- a * pnorm(x, mean = mu1, sd = sigma1) + b * pnorm(x, mean = mu2, sd = sigma2)

Quantile

In [ ]:
p <- seq(0, 1, length=100)
quantiles <- quantile(mixture_data, p)

Plot

In [ ]:
value <- quantilplot("Summe von Normalverteilungen", x, pdf, cdf, p, quantiles)